# Network Tool - Unofficial Role Identifier

### Context

Research, e.g. [Kraut et al](http://kraut.hciresearch.org/sites/kraut.hciresearch.org/files/open/kraut90-InformalCommInOrgs.pdf), indicates that informal organisational-networks play an important role in facilitating productive and efficient organisational processes. As these networks are not defined within a clear organisational hierarchy, however, developing an understanding of how staff interact informally is challenging to achieve. This tool seeks to clarify this uncertainty, through the utilization of interaction-based data, to identify the informal roles that staff appear to be occupying.

Building on research by [Harris & Nelson](https://books.google.co.uk/books/about/Applied_Organizational_Communication.html?id=9s6OAgAAQBAJ&redir_esc=y), this tool was designed to identify five key informal roles within staff networks: 

* Organizer - This role is concerned with individuals that link different groups across organisational-networks. Project-managers are a quintessential example of this role, as these individuals often traverse numerous organisational-communities in order to spread information and manage tasks across project-networks.


* Gatekeeper - Gatekeepers are the individuals that facilitate and mediate an effective flow of information across communication-networks. If a teams delivery-manager, for example, is the only individual within that team frequently interacting with a wider organisational network, this indivdual would act as the gatekeeper of information to their team.


* Stars - These individuals are at the heart of organisational communities, and often have significant influence with fellow community-members. Stars can potentially be team-managers, however these individuals may also not have a determined official role. Essentially a star is an individual with strong community-presence, whether that presence if formal or informal.


* Isolates - As the name suggests, these are individuals who do not need to communicate within a wider organisational-network to carry out their work. A contractor who only converses with a specific team-member, for example, can be regarded as an isolate. These individuals are important to identify, as they are likely last to be reached by information flowing through a network.


* Members - Group-members are individuals that are non-identifiable in the context of the other roles listed, and individually do not have as much potential to impact network efficiency like other identified roles.





### Methodology

The Enron email-corpus was used for constructing this tool, as to my knowledge this is the only open-source dataset containing comprehensive information relating to internal organisational communication. This dataset contains around 500,000 emails from Enron staff between 1998-2002, and was released as part of a fraud investigation of Enron after its liquidation in 2002. 

The tools functioning is based on a variety of metrics relevant to network-science, which can be used to evaluate the importance and characteristics of actors within constructed networks. Before development, an extensive period of time was dedicated to examining which particular metrics would be most informative in the context of this research. Research by [Schwartz and Jacobson](https://www.sciencedirect.com/science/article/pii/0030507377900265), for example, helped contextualize the organisational importance of the roles mentioned above, while lectures from [DH Chau](http://poloclub.gatech.edu/cse6242/2014spring/lectures/CSE6242-20140218-GraphsAlgoApp.pdf) helped inform which network-based metrics would be most appropriate for role-identification. 

The tool is Python-based, and utilizes packages which specialize in network-science. NetworkX, especially, provided a package which contained a multitude of different metrics and techniques utilized by this research. 


### Uses

This tool is warranted, as it can assist in answering key questions regarding organizational dynamics, including:

* Who are the key players in the network, officially and informally?

* Which staff have more of an influence on internal-communication networks than others?

* Is information within the network spreading efficiently?

Essentially, this tool seeks to provide decision-makers with a data-driven understanding of organisational dynamics, which can be used to build on their current understanding of how organisational interaction is taking place. Therefore, this tool should not be considered as a replacement of an organisational map, but as a point of reference to evaluate if current organisational process is effective.

## Workings

The functioning of this tool will now be discussed in terms of the techniques used to identify informal roles, and how the output of these identifications link together. A detailed discussion of tool parameters, such as choice of algorithms, can be found after the code section.

#### Packages & Input-Data

1. Import packages
2. Import graph data


Four packages were used for construction of the tool, networkx, community, pandas, and operator. The former packages were used for network-based functioning, while the latter packages are involved in more general functioning throughout the script.


Data to be used with this tool should be formatted in the following way: Source Node, Target Node, Weight. Source and Target refer to who is sending and receiving the emails, whereas Weight refers to the quantity of emails between two particular individuals. Interactions should not be combined for both nodes, but instead seperated into the formats of a>b, and b<a.

In [ ]:
import networkx as nx
from community import community_louvain
import pandas as pd
import operator
G = nx.read_weighted_edgelist('Only_50_Employees1.csv', delimiter=',', create_using = nx.DiGraph(), nodetype=str)

#### Organizers

1. Run organizer measures


Initial research identified several key metrics which could help identify organizers within networks: 


* Betweenness-centrality: A measure of how many of the shortest paths in a network flow through a particular node. If a node has high betweenness-centrality, it indicates that this node is very central in the network structure, and is therefore more likely to be an interaction hub.


* Eigenvector Centrality: A measure of a nodes number of connections, which more weight given to nodes whose connections are to other important nodes. This metric is useful, as organizers are likely to interact extensivley with other hub-based nodes, such as senior-managers or delivery-managers.


* PageRank: This measure was created by Google, and is used to rank the importance of websites which appear in Google search results. Essentially, the algorithm uses a probability distribution to represent the likelihood that a given signal travelling through a network will end up at a particular node, with a nodes score determined as the probability of landing there if the signal was randomly stopped. 

In [ ]:
betweenness_score = dict(nx.betweenness_centrality(G))
eigen_score = dict(nx.eigenvector_centrality(G))
page_score = dict(nx.pagerank(G))

As each of these metrics can face pitfalls, especially in the context of local minima, it was decided to combine the three into an overall score, which together would be less susceptible to minima than one metric alone.

1. Integrate the three metric values into one dataframe
2. To avoid scale-bias, standardize all values and turn positive by adding 1
3. Combining the standardized metrics into one overall score
4. Return this overall score to a dictionary

In [ ]:
mydicts = [page_score, betweenness_score, eigen_score]
df = pd.concat([pd.Series(d) for d in mydicts], axis=1).fillna(0).T
df.index = ['page_score', 'betweenness_score', 'eigen_score']
df = df.transpose()
del page_score, eigen_score, betweenness_score, mydicts
df = (df - df.mean()) / (df.max() - df.min())
minus_columns = ['page_score', 'betweenness_score', 'eigen_score']
df = df[minus_columns] + 1
df['score'] = df['page_score'] + df['betweenness_score'] + df['eigen_score']
del df['page_score'], df['betweenness_score'], df['eigen_score']
score_dict = df['score'].to_dict()

To allow for scalability, a percentage function was created which returns the top x% of the highest scoring nodes in the organizer dictionary. The value is currently set to 10%, however this can be changed to suit organisational practice, or the size of the network.

1. Create percentage function
2. Sort dictionary and return top 10%
3. Appoint determined nodes with a score of 0, to indicate organizer

In [ ]:
n = int(len(score_dict) * 0.10)
organizer_dict = dict(sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)[:n])
organizer_dict = {x: 0 for x in organizer_dict}
del score_dict, df, n, minus_columns

#### Gatekeepers

The networkx 'bridges' function was used to find gatekeepers, as this returns the links within the network which if removed would result in the network splitting into different sub-structures. The function returns the format a>b, with a being the node connected to the larger sub-structure.

1. Find bridges in the network
2. Remove node b for all bridges (the more isolated node)
3. Remove all nodes already found for the organizer role
4. Create a code of 1 to identify gatekeepers by

In [ ]:
G = nx.read_weighted_edgelist('Only_50_Employees1.csv', delimiter=',', create_using = nx.Graph(), nodetype=str)
gatekeeper = dict(nx.bridges(G))
gatekeeper_dict = {k: v for k, v in gatekeeper.items() if k not in organizer_dict}
gatekeeper_dict = {x: 1 for x in gatekeeper_dict}
del gatekeeper

#### Stars
To find stars, the overall graph was partitioned using a community detection algorithm (See Appendix 1.1), and then the node with the largest number of links for each community is returned.

1. Partition communities into disconnected sub-graphs
2. Find the node with the highest degree for each community
3. Incorporate all highest-degree nodes into a dictionary
4. Remove nodes already identified in the organizer and gatekeeper sections

In [ ]:
part = community_louvain.best_partition(G)                                                     
def invert(d):                                          
    r = {}
    for k, v in d.items():
        r.setdefault(v, []).append(k)
    return r
invert_partition = invert(part)
star_dict = {}        # iterate over each community
for community_id in invert_partition.keys():                                                   
    temp_graph = G.subgraph(invert_partition[community_id])
    temp_degree = dict(temp_graph.degree())                                                   
    star_dict[community_id] = max(temp_degree, key=lambda x: temp_degree[x])                   
star_dict = dict((v,k) for k,v in sorted(star_dict.items(), key=operator.itemgetter(1)))
star_dict = {k: v for k, v in star_dict.items() if k not in organizer_dict}
star_dict = {k: v for k, v in star_dict.items() if k not in gatekeeper_dict}
star_dict = {x: 2 for x in star_dict}
del community_id, invert_partition, part, temp_degree

#### Isolates
Isolated nodes were found by creating a function which creates a dictionary of all nodes with <= 1 link in the network.

1. Find the degree for all network nodes
2. Remove all nodes with a degree of <= 1

In [ ]:
isolate_dict = dict(G.degree())
isolate_dict = {key:val for key, val in isolate_dict.items() if val == 1 or 0}
isolate_dict = {x: 3 for x in isolate_dict}

#### Final Merge

Lastly, a function is used to combine all returned roles into one dictionary.

In [ ]:
final_roles = {**organizer_dict, **gatekeeper_dict, **star_dict, **isolate_dict}
del organizer_dict, gatekeeper_dict, star_dict, isolate_dict

#### Appendix 1.1 - Community Detection Algorithms
Numerous algorithms exist which could be used for community detection within the Star section, a full list of which can be found [here](http://igraph.org/c/doc/igraph-Community.html). To determine the most appropriate partioning method, this research aimed to optimize graph modularity, which is essentially a measure of how inter-related communication is among different groups within a network.

Iterativley, each community-detection algorithm was implemented, and the modularity score for each was recorded. Additionally, the amount of different partitions each module produced was also noted, which ranged from 4 to 13. Overall, it was determined that the Louvain method was the best-match for the tool, for several reasons. Primarily, the Louvain method returned the highest modularity score for all algorithms tests (0.543 for Louvain, 0.442 for nearest algorithm). Secondly, Louvain was also selected as it produced a good amount of communities relative to the small dataset(6), while other methods were producing up to 13.

#### Appendix 1.2 - Information regarding algorithms used

[Betweenness Centrality](https://en.wikipedia.org/wiki/Betweenness_centrality)
    
[PageRank](https://en.wikipedia.org/wiki/PageRank)
    
[Eigenvector](https://en.wikipedia.org/wiki/Eigenvector_centrality)
    
[Modularity](https://en.wikipedia.org/wiki/Modularity_(networks)

[Louvain](https://en.wikipedia.org/wiki/Louvain_Modularity)